In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime

import utils.models as models
import utils.plotting as plotting
import utils.dataloaders as dl
import utils.traintest as tt
import utils.adversarial as adv
import utils.eval as ev
import utils.gmm_helpers as gmm_helpers
import model_params as params

from importlib import reload

import utils.mc_dropout as mc

import argparse


#parser = argparse.ArgumentParser(description='Define hyperparameters.', prefix_chars='-')

#parser.add_argument('--gpu', type=int, default=0, help='GPU index.')
#parser.add_argument('--dataset', type=str, default='MNIST', help='MNIST, FMNIST, SVHN, CIFAR10, CIFAR100')


#hps = parser.parse_args()


dataset = 'MNIST'

saving_string = dataset + '_base'
device = torch.device('cuda:1')

model_params = params.params_dict[dataset](augm_flag=True)

def train(model, device, train_loader, optimizer, epoch, 
          verbose=100, noise_loader=None, epsilon=.3):
    
    criterion = nn.NLLLoss()
    model.train()
    
    train_loss = 0
    correct = 0
    

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        output = F.log_softmax(model(data), dim=1)
        #output = model(data)
        
        loss = criterion(output, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = output.max(1)
        correct += predicted.eq(target).sum().item()
        if (batch_idx % verbose == 0) and verbose>0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    return train_loss/len(train_loader.dataset), correct/len(train_loader.dataset)


if dataset=='MNIST':
    model = mc.LeNet()
elif dataset=='FMNIST':
    model = mc.vgg13(in_channels=1, num_classes=10)
elif dataset in ['SVHN', 'CIFAR10']:
    model = mc.vgg13(in_channels=3, num_classes=10)
elif dataset=='CIFAR100':
    model = mc.vgg13(in_channels=3, num_classes=100)


model = model.to(device)


param_groups = [{'params':model.parameters(),'lr':model_params.lr, 'weight_decay':5e-4}]
    
if dataset=='MNIST':
    optimizer = optim.Adam(param_groups)
else: 
    optimizer = optim.SGD(param_groups, momentum=0.9)


for epoch in range(100):
    if epoch+1 in [50,75,90]:
        for group in optimizer.param_groups:
            group['lr'] *= .1
 
    trainloss, correct_train = train(model, device, model_params.train_loader,  
                                     optimizer, epoch, verbose=-1)
    print(str(epoch) + ': \t' + str(correct_train))

model = model.to('cpu')

mc_model = mc.MC_Model(model, iterations=10, classes=model_params.classes)

torch.save(mc_model, 'SavedModels/other/mcdo/' + dataset + '.pth')

KeyboardInterrupt: 

In [3]:
reload(mc)
mc_model = mc.MC_Model(model, iterations=7, classes=model_params.classes)
mc_model.cpu()

MC_Model(
  (model): LeNet(
    (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (fc1): Linear(in_features=3136, out_features=1024, bias=True)
    (fc2): Linear(in_features=1024, out_features=10, bias=True)
    (dropout): MC_dropout(p=0.5)
  )
)

In [8]:
torch.save(mc_model, 'SavedModels/other/mcdo/' + dataset + '.pth')

tensor(0.4970)

In [5]:
x = torch.rand(10, 1, 28, 28)

In [10]:
(1000.*mc_model(x)).exp()

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9563,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9895,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5617,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9698,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9924,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.7467,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9423,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9794,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9213,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9823,
         0.0000]], grad_fn=<

In [50]:
data = next(iter(model_params.test_loader))[0]
out_data = next(iter(model_params.loaders[0][1]))[0]

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime

import utils.models as models
import utils.plotting as plotting
import utils.dataloaders as dl
import utils.traintest as tt
import utils.adversarial as adv
import utils.eval as ev
import utils.gmm_helpers as gmm_helpers
import model_params as params

from importlib import reload

import utils.mc_dropout as mc

import argparse


#parser = argparse.ArgumentParser(description='Define hyperparameters.', prefix_chars='-')

#parser.add_argument('--gpu', type=int, default=0, help='GPU index.')
#parser.add_argument('--dataset', type=str, default='MNIST', help='MNIST, FMNIST, SVHN, CIFAR10, CIFAR100')


#hps = parser.parse_args()

import utils.mc_dropout as mcdo
dataset = 'MNIST'


In [3]:
model_orig = torch.load('SavedModels/other/mcdo/' + dataset + '.pth')

/usr/local/lib/python3.5/dist-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'utils.mc_dropout.MC_Model' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.5/dist-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'utils.mc_dropout.LeNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [5]:
base_model = mcdo.LeNet()
base_model.load_state_dict(model_orig.model.state_dict())

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [23]:
model = mcdo.MC_Model_logit(base_model)

In [25]:
model(torch.rand(10,1,28,28)).max(1)[0]

tensor([-0.3928, -0.3599, -0.5513, -0.3469, -0.3948, -0.2159, -0.3062, -0.6145,
        -0.5313, -0.3590], grad_fn=<MaxBackward0>)

In [8]:
data = next(iter(dl.MNIST(train=False)))[0]

In [26]:
model(data).max(1)[0]

tensor([-0.4369, -1.0899, -0.2506, -0.5487, -0.6325, -0.5732, -0.6794, -0.5849,
        -0.5577, -1.2460, -1.1434, -1.0284, -0.6601, -0.6757, -0.5585, -1.0862,
        -0.7475, -0.5529, -0.6506, -0.8098, -0.5321, -1.2555, -1.3143, -0.6279,
        -0.4335, -0.5435, -0.5489, -1.6638, -0.7860, -0.4119, -1.2742, -0.7531,
        -1.3202, -0.5019, -0.6100, -0.4552, -0.6903, -0.6868, -0.5638, -0.6917,
        -0.1884, -0.4826, -1.2094, -0.3916, -1.1517, -0.6226, -0.5491, -0.6264,
        -0.5130, -0.9235, -0.6805, -0.7852, -1.0110, -0.5753, -0.8085, -1.3219,
        -1.3782, -0.4699, -0.8234, -0.4747, -1.0878, -1.2073, -0.2912, -1.2442,
        -0.7350, -0.2809, -0.7515, -0.2932, -0.9785, -0.7294, -0.9911, -0.5688,
        -0.6702, -0.7228, -0.3941, -0.8313, -0.9419, -0.4428, -0.5547, -0.4639,
        -0.4419, -1.1334, -1.2879, -0.5020, -0.9558, -0.7522, -0.3997, -0.5230,
        -0.5627, -0.5192, -1.8083, -0.6673, -0.3194, -0.8874, -0.3906, -0.8105,
        -0.4415, -1.0257, -1.0334, -0.68

In [33]:
device = torch.device('cuda:2')

In [27]:
def forward(self, x):
        out = []
        for _ in range(self.iterations):
            out.append(self.model.logit(x))
        out = torch.stack(out)
        y = out.mean(0)
        print(((out - y[None,:,:])**2).shape)
        uncertainty = ((out - y[None,:,:])**2).mean(0).mean(1)
        return uncertainty

In [35]:
model.to(device);

In [31]:
model_params = params.params_dict[dataset](augm_flag=True)

In [36]:
df = ev.evaluate(model, device, dataset, model_params.loaders, load_adversaries=False, 
                 writer=None, epoch=0, drop_mmc=True)
print(df)

                    AUC
DataSet                
FMNIST        12.456792
EMNIST        31.437948
GrayCIFAR10    2.974106
Noise          0.085721
UniformNoise  25.758609
